In [1]:

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
import tensorflow as tf
from pandas import Series
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model
from keras.layers import Bidirectional
from keras.layers import Input

import tensorflow as tf
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.preprocessing import sequence

# fix random seed for reproducibility
tf.random.set_seed(7)

# TARGETS

In [2]:
df = pd.read_csv(f'../../db/data/merge/secondary/logs_.csv', index_col=0)
df.reset_index(inplace=True, drop=True)

df

,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
0,-0.056357,0.044868,-0.236804,-0.027523,-0.191681,0.090711,-0.187507,-0.277763,-0.046156,-0.119630,...,-0.147436,0.041166,-0.099046,0.009273,0.090472,-0.190166,-0.139295,0.145630,0.228210,-0.048759
1,0.057418,0.116563,-0.162348,-0.068844,-0.031983,0.047432,-0.211379,-0.267380,-0.023087,0.033756,...,-0.180774,-0.117932,-0.052168,0.159828,0.212410,-0.054403,0.069672,0.087273,0.144556,-0.126891
2,-0.206473,0.062703,-0.262812,-0.018366,-0.243434,0.294601,-0.058796,-0.346427,0.000000,-0.215034,...,-0.255167,-0.003746,-0.240322,-0.190002,0.082882,-0.245215,-0.223152,0.033227,0.323370,-0.029303
3,-0.373553,-0.049272,0.125235,-0.404915,-0.733783,0.309505,0.504518,0.201297,0.063483,-0.320355,...,0.300982,-0.251275,-0.561166,0.052171,0.367330,0.544108,-0.307650,0.348187,0.669742,0.846226
4,-0.036356,-0.236578,-0.062598,0.101392,0.012918,-0.200263,-0.070237,0.133973,0.184438,0.191093,...,0.040733,0.270002,0.152371,-0.147307,-0.350272,-0.203694,-0.098087,-0.024836,-0.256038,-0.111437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111091,-0.111104,-0.019367,-0.046219,-0.060675,-0.152940,0.088497,0.060510,-0.024679,0.303152,0.193995,...,0.078071,0.051580,-0.035225,-0.039843,0.053424,0.017658,-0.083248,0.045766,0.133986,0.100968
111092,-0.111104,-0.019367,-0.046219,-0.060675,-0.152940,0.088497,0.060510,-0.024679,0.303152,0.193995,...,0.078071,0.051580,-0.035225,-0.039843,0.053424,0.017658,-0.083248,0.045766,0.133986,0.100968
111093,-0.111104,-0.019367,-0.046219,-0.060675,-0.152940,0.088497,0.060510,-0.024679,0.303152,0.193995,...,0.078071,0.051580,-0.035225,-0.039843,0.053424,0.017658,-0.083248,0.045766,0.133986,0.100968
111094,-0.111104,-0.019367,-0.046219,-0.060675,-0.152940,0.088497,0.060510,-0.024679,0.303152,0.193995,...,0.078071,0.051580,-0.035225,-0.039843,0.053424,0.017658,-0.083248,0.045766,0.133986,0.100968


In [3]:
#convert logs to classification states = ( 1, 0, -1 ) 

def condition(x, threshold):
    if x>threshold:
        return 1
    elif x<-threshold:
        return -1
    else:
        return 0

df_ = df.copy()

for sym in df.columns:
    df_[sym] = df[sym].map( lambda x: condition(x, 0.05) )

df_.reset_index(inplace=True, drop=True)

print(df_)

y = []
for i in range(len(df)):
    y.append(np.array(df.iloc[i].values))

y = np.array(y)

print(y)

df_


        AUD_CAD  AUD_CHF  AUD_JPY  AUD_NZD  AUD_USD  CAD_CHF  CAD_JPY  \
0            -1        0       -1        0       -1        1       -1   
1             1        1       -1       -1        0        0       -1   
2            -1        1       -1        0       -1        1       -1   
3            -1        0        1       -1       -1        1        1   
4             0       -1       -1        1        0       -1       -1   
...         ...      ...      ...      ...      ...      ...      ...   
111091       -1        0        0       -1       -1        1        1   
111092       -1        0        0       -1       -1        1        1   
111093       -1        0        0       -1       -1        1        1   
111094       -1        0        0       -1       -1        1        1   
111095       -1        0        0       -1       -1        1        1   

        CHF_JPY  EUR_AUD  EUR_CAD  ...  GBP_JPY  GBP_NZD  GBP_USD  NZD_CAD  \
0            -1        0       -1  ...       

,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
0,-1,0,-1,0,-1,1,-1,-1,0,-1,...,-1,0,-1,0,1,-1,-1,1,1,0
1,1,1,-1,-1,0,0,-1,-1,0,0,...,-1,-1,-1,1,1,-1,1,1,1,-1
2,-1,1,-1,0,-1,1,-1,-1,0,-1,...,-1,0,-1,-1,1,-1,-1,0,1,0
3,-1,0,1,-1,-1,1,1,1,1,-1,...,1,-1,-1,1,1,1,-1,1,1,1
4,0,-1,-1,1,0,-1,-1,1,1,1,...,0,1,1,-1,-1,-1,-1,0,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111091,-1,0,0,-1,-1,1,1,0,1,1,...,1,1,0,0,1,0,-1,0,1,1
111092,-1,0,0,-1,-1,1,1,0,1,1,...,1,1,0,0,1,0,-1,0,1,1
111093,-1,0,0,-1,-1,1,1,0,1,1,...,1,1,0,0,1,0,-1,0,1,1
111094,-1,0,0,-1,-1,1,1,0,1,1,...,1,1,0,0,1,0,-1,0,1,1


# FEATURES

In [4]:
# load features

df2 = pd.read_csv('../../db/data/merge/tendency/tendency.csv', index_col=0)
df2.reset_index(inplace=True, drop=True)

df2.columns


Index(['AUD_CAD', 'AUD_CHF', 'AUD_JPY', 'AUD_NZD', 'AUD_USD', 'CAD_CHF',
       'CAD_JPY', 'CHF_JPY', 'EUR_AUD', 'EUR_CAD', 'EUR_CHF', 'EUR_GBP',
       'EUR_JPY', 'EUR_NZD', 'EUR_USD', 'GBP_AUD', 'GBP_CAD', 'GBP_CHF',
       'GBP_JPY', 'GBP_NZD', 'GBP_USD', 'NZD_CAD', 'NZD_CHF', 'NZD_JPY',
       'NZD_USD', 'USD_CAD', 'USD_CHF', 'USD_JPY', 'AUD_low', 'CAD_low',
       'CHF_low', 'EUR_low', 'GBP_low', 'JPY_low', 'NZD_low', 'USD_low',
       'AUD_high', 'CAD_high', 'CHF_high', 'EUR_high', 'GBP_high', 'JPY_high',
       'NZD_high', 'USD_high', 'AUD_rsi', 'CAD_rsi', 'CHF_rsi', 'EUR_rsi',
       'GBP_rsi', 'JPY_rsi', 'NZD_rsi', 'USD_rsi', 'AUD_ema', 'CAD_ema',
       'CHF_ema', 'EUR_ema', 'GBP_ema', 'JPY_ema', 'NZD_ema', 'USD_ema',
       'AUD_sharpe', 'CAD_sharpe', 'CHF_sharpe', 'EUR_sharpe', 'GBP_sharpe',
       'JPY_sharpe', 'NZD_sharpe', 'USD_sharpe', 'month_sin', 'month_cos',
       'day_sin', 'day_cos', 'weekday_sin', 'weekday_cos', 'hour_sin',
       'hour_cos'],
      dtype='object

In [5]:
# scaling features

def scale ( data ):

    series = Series(data)
    # prepare data for normalization
    values = series.values
    values = values.reshape((len(values), 1))
    # train the normalization
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(values)
    #print('Min: %f, Max: %f' % (scaler.data_min_, scaler.data_max_))
    # normalize the dataset and print
    normalized = scaler.transform(values)
    #print(normalized)
    # inverse transform and print
    inversed = scaler.inverse_transform(normalized)
    #print(inversed)

    return normalized, inversed


df2.columns

Index(['AUD_CAD', 'AUD_CHF', 'AUD_JPY', 'AUD_NZD', 'AUD_USD', 'CAD_CHF',
       'CAD_JPY', 'CHF_JPY', 'EUR_AUD', 'EUR_CAD', 'EUR_CHF', 'EUR_GBP',
       'EUR_JPY', 'EUR_NZD', 'EUR_USD', 'GBP_AUD', 'GBP_CAD', 'GBP_CHF',
       'GBP_JPY', 'GBP_NZD', 'GBP_USD', 'NZD_CAD', 'NZD_CHF', 'NZD_JPY',
       'NZD_USD', 'USD_CAD', 'USD_CHF', 'USD_JPY', 'AUD_low', 'CAD_low',
       'CHF_low', 'EUR_low', 'GBP_low', 'JPY_low', 'NZD_low', 'USD_low',
       'AUD_high', 'CAD_high', 'CHF_high', 'EUR_high', 'GBP_high', 'JPY_high',
       'NZD_high', 'USD_high', 'AUD_rsi', 'CAD_rsi', 'CHF_rsi', 'EUR_rsi',
       'GBP_rsi', 'JPY_rsi', 'NZD_rsi', 'USD_rsi', 'AUD_ema', 'CAD_ema',
       'CHF_ema', 'EUR_ema', 'GBP_ema', 'JPY_ema', 'NZD_ema', 'USD_ema',
       'AUD_sharpe', 'CAD_sharpe', 'CHF_sharpe', 'EUR_sharpe', 'GBP_sharpe',
       'JPY_sharpe', 'NZD_sharpe', 'USD_sharpe', 'month_sin', 'month_cos',
       'day_sin', 'day_cos', 'weekday_sin', 'weekday_cos', 'hour_sin',
       'hour_cos'],
      dtype='object

In [6]:

df2.replace([np.inf, -np.inf], 0, inplace=True)

X = []
for i in range(len(df2)):
    X.append(np.array(df2.iloc[i].values))

X = np.array(X)

print(X)

df2.shape


[[-0.0563575   0.0448682  -0.2368045  ...  1.          0.
   1.        ]
 [ 0.0574181   0.11656319 -0.16234783 ...  1.          0.25881905
   0.96592583]
 [-0.20647311  0.06270295 -0.26281224 ...  1.          0.5
   0.8660254 ]
 ...
 [-0.11110372 -0.01936671 -0.04621946 ...  0.6234898  -0.70710678
   0.70710678]
 [-0.11110372 -0.01936671 -0.04621946 ...  0.6234898  -0.5
   0.8660254 ]
 [-0.11110372 -0.01936671 -0.04621946 ...  0.6234898  -0.25881905
   0.96592583]]


(111096, 76)

In [7]:
X_train = X[:int(len(X)*0.5)]
y_train = y[:int(len(X)*0.5)]

X_train.shape , y_train.shape

((55548, 76), (55548, 28))

In [8]:
np.unique(y_train)

array([-5.25684051, -5.13585269, -4.9187617 , ...,  6.10790881,
        6.11082465,  6.55189373])

In [9]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
		X.append(seq_x)
		y.append(seq_y)

	return np.array(X), np.array(y)

In [10]:
input_layer = Input(shape=(600,6,))

encoder = Bidirectional(
    LSTM(250),
    merge_mode='concat',
    return_sequences=True
)(input_layer)
decoder = LSTM(250, return_sequences=True)(encoder)
pred = TimeDistributed(Dense(1))(decoder)
model = Model(inputs=input_layer, outputs=pred)
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

model.fit(X_train, y_train, epochs = 20, batch_size = 32)

TypeError: ('Keyword argument not understood:', 'return_sequences')

In [ ]:

X__ = []
for i in X.columns:
    normalized, _ = scale( X_train[i] )
    X__.append(normalized)

X__ = np.array(X__)
X__ = X__.reshape(4027848, 1)

X__



array([[ 0.0988461 ],
       [ 0.10489588],
       [ 0.05442634],
       ...,
       [-0.25881905],
       [-0.5       ],
       [-0.70710678]])

In [ ]:
y__ = []
for i in y.columns:
    normalized, _ = scale( y_train[i] )
    y__.append(normalized)

y__ = np.array(y__)
y__ = y__.reshape(1483944, 1)

y__


AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [ ]:
y.shape[1]

28

In [ ]:
X__

NameError: name 'X__' is not defined

In [ ]:
n_lag = 1
n_neurons = 200
n_epochs = 1000
n_batch = 1
n_neurons = 1

# design network
model = Sequential()

model.add(LSTM(n_neurons, stateful=True))

model.add(Dense(y.shape[1]))

model.compile(optimizer='adam', loss='mse', metrics='accuracy')


MAX_EPOCHS = 10000

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError(), tf.keras.metrics.Accuracy()])

  history = model.fit(X=X__['train'], y=y__['train'], epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  
  return history



In [15]:
y_ = df['EUR_USD'].map( lambda x: condition(x, 0.05) )[10000:-100000]
X_ = df2.filter(regex=ccy).[10000:-100000]

y_ = np.array(y_)



X_.shape , y_.shape

((100096, 2), (100096,))

In [16]:
y_

array([ 0, -1,  1, ...,  0,  1,  1])

In [17]:
from keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse = False)

y_cat = encoder.fit_transform(y_.reshape(-1,1))
y_cat.shape

(100096, 3)

In [18]:
X_.shape , y_cat.shape

((100096, 2), (100096, 3))

In [19]:
X_expanded = np.expand_dims(X_ , -1)
X_expanded.shape

(100096, 2, 1)

In [20]:
EPOCHS = 200 
from tensorflow.keras import layers

# define model
def initialize_model(X):
    model = Sequential()
    
    model.add(layers.Masking(mask_value = -1 , input_shape = (X_.shape[1],1)))
    model.add(layers.GRU(units=20 , activation = 'tanh',return_sequences = True))
    model.add(layers.GRU(units=20 , activation = 'tanh' , return_sequences = False))
    model.add(layers.Dense(50 , activation = 'relu'))
    #model.add(LSTM(50, activation='relu', input_shape=(X_.shape[1], 10)))
    model.add(Dense(3, activation='softmax',))


    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

X_expanded = np.expand_dims(X_ , -1)
model = initialize_model(X_expanded)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (None, 2, 1)              0         
                                                                 
 gru (GRU)                   (None, 2, 20)             1380      
                                                                 
 gru_1 (GRU)                 (None, 20)                2520      
                                                                 
 dense (Dense)               (None, 50)                1050      
                                                                 
 dense_1 (Dense)             (None, 3)                 153       
                                                                 
Total params: 5,103
Trainable params: 5,103
Non-trainable params: 0
_________________________________________________________________


In [21]:
# fit model
model.fit(X_expanded, y_cat, epochs=EPOCHS)

Epoch 1/200
3128/3128 [==============================] - 39s 9ms/step - loss: 1.0102 - accuracy: 0.5281
Epoch 2/200
3128/3128 [==============================] - 28s 9ms/step - loss: 1.0069 - accuracy: 0.5282
Epoch 3/200
3128/3128 [==============================] - 28s 9ms/step - loss: 1.0069 - accuracy: 0.5282
Epoch 4/200
1383/3128 [============>.................] - ETA: 16s - loss: 1.0052 - accuracy: 0.5299

KeyboardInterrupt: 

In [22]:
# define model
def initialize_model(X):
    model = Sequential()
    
    model.add(layers.Masking(mask_value = -1 , input_shape = (X_.shape[1],1)))
    model.add(LSTM(50, activation='relu'))
    model.add(layers.Dense(50 , activation = 'relu'))
    model.add(Dense(3, activation='softmax',))


    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

X_expanded = np.expand_dims(X_ , -1)
model = initialize_model(X_expanded)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_1 (Masking)         (None, 2, 1)              0         
                                                                 
 lstm_1 (LSTM)               (None, 50)                10400     
                                                                 
 dense_2 (Dense)             (None, 50)                2550      
                                                                 
 dense_3 (Dense)             (None, 3)                 153       
                                                                 
Total params: 13,103
Trainable params: 13,103
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.fit(X_expanded, y_cat, epochs=EPOCHS)

Epoch 1/200
3128/3128 [==============================] - 21s 6ms/step - loss: 1.6105 - accuracy: 0.5279
Epoch 2/200
1641/3128 [==============>...............] - ETA: 8s - loss: 1.0054 - accuracy: 0.5292

KeyboardInterrupt: 

In [24]:
def initialize_model(X):
    model = Sequential()
    
    model.add(LSTM(50, activation='relu' , input_shape = (X.shape[1] , 1)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(50 , activation = 'relu'))
    model.add(Dense(3, activation='softmax',))


    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

X_expanded = np.expand_dims(X_ , -1)

model = initialize_model(X_)

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 50)                10400     
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense_4 (Dense)             (None, 50)                2550      
                                                                 
 dense_5 (Dense)             (None, 3)                 153       
                                                                 
Total params: 13,103
Trainable params: 13,103
Non-trainable params: 0
_________________________________________________________________


In [25]:
X_.shape

(100096, 2)

In [26]:
model.fit(X_, y_cat, epochs=EPOCHS)

Epoch 1/200
3128/3128 [==============================] - 18s 5ms/step - loss: 955.3786 - accuracy: 0.5279
Epoch 2/200
3128/3128 [==============================] - 16s 5ms/step - loss: 1352.8204 - accuracy: 0.5281
Epoch 3/200
3128/3128 [==============================] - 15s 5ms/step - loss: 1158.3273 - accuracy: 0.5280
Epoch 4/200
3128/3128 [==============================] - 16s 5ms/step - loss: 1288.3821 - accuracy: 0.5282
Epoch 5/200
3128/3128 [==============================] - 15s 5ms/step - loss: 2508.6672 - accuracy: 0.5283
Epoch 6/200
3128/3128 [==============================] - 16s 5ms/step - loss: 417.0493 - accuracy: 0.5280
Epoch 7/200
3128/3128 [==============================] - 16s 5ms/step - loss: 1081.5441 - accuracy: 0.5281
Epoch 8/200
3128/3128 [==============================] - 16s 5ms/step - loss: 1770.8926 - accuracy: 0.5281
Epoch 9/200
2572/3128 [=======================>......] - ETA: 2s - loss: 1110.0001 - accuracy: 0.5279

KeyboardInterrupt: 

In [119]:
X.shape ,y.shape

((111096, 76), (111096, 28))

In [41]:
encoder = OneHotEncoder(sparse = False)

[['AUD_CAD', 'AUD_CHF', 'AUD_JPY', 'AUD_NZD', 'AUD_USD', 'CAD_CHF',
       'CAD_JPY', 'CHF_JPY', 'EUR_AUD', 'EUR_CAD', 'EUR_CHF', 'EUR_GBP',
       'EUR_JPY', 'EUR_NZD', 'EUR_USD', 'GBP_AUD', 'GBP_CAD', 'GBP_CHF',
       'GBP_JPY', 'GBP_NZD', 'GBP_USD', 'NZD_CAD', 'NZD_CHF', 'NZD_JPY',
       'NZD_USD', 'USD_CAD', 'USD_CHF', 'USD_JPY', 'AUD_low', 'CAD_low',
       'CHF_low', 'EUR_low', 'GBP_low', 'JPY_low', 'NZD_low', 'USD_low',
       ]]
       
y_ = df.applymap( lambda x: condition(x, 0.05) )[1000:-10000]
X_ = df2[[ 'AUD_low', 'CAD_low',
       'CHF_low', 'EUR_low', 'GBP_low', 'JPY_low', 'NZD_low', 'USD_low',
       'AUD_high', 'CAD_high', 'CHF_high', 'EUR_high', 'GBP_high', 'JPY_high',
       'NZD_high', 'USD_high', 'AUD_rsi', 'CAD_rsi', 'CHF_rsi', 'EUR_rsi',
       'GBP_rsi', 'JPY_rsi', 'NZD_rsi', 'USD_rsi', 'AUD_ema', 'CAD_ema',
       'CHF_ema', 'EUR_ema', 'GBP_ema', 'JPY_ema', 'NZD_ema', 'USD_ema',
       'AUD_sharpe', 'CAD_sharpe', 'CHF_sharpe', 'EUR_sharpe', 'GBP_sharpe',
       'JPY_sharpe', 'NZD_sharpe', 'USD_sharpe', 'month_sin', 'month_cos',
       'day_sin', 'day_cos', 'weekday_sin', 'weekday_cos', 'hour_sin',
       'hour_cos']][1000:-10000]

y_ = np.array(y_)
y_.shape , X_.shape

((100096, 28), (100096, 48))

In [49]:
stable = [ 'AUD_low', 'CAD_low',
       'CHF_low', 'EUR_low', 'GBP_low', 'JPY_low', 'NZD_low', 'USD_low',
       'AUD_high', 'CAD_high', 'CHF_high', 'EUR_high', 'GBP_high', 'JPY_high',
       'NZD_high', 'USD_high', 'AUD_rsi', 'CAD_rsi', 'CHF_rsi', 'EUR_rsi',
       'GBP_rsi', 'JPY_rsi', 'NZD_rsi', 'USD_rsi', 'AUD_ema', 'CAD_ema',
       'CHF_ema', 'EUR_ema', 'GBP_ema', 'JPY_ema', 'NZD_ema', 'USD_ema',
       'AUD_sharpe', 'CAD_sharpe', 'CHF_sharpe', 'EUR_sharpe', 'GBP_sharpe',
       'JPY_sharpe', 'NZD_sharpe', 'USD_sharpe', 'month_sin', 'month_cos',
       'day_sin', 'day_cos', 'weekday_sin', 'weekday_cos', 'hour_sin',
       'hour_cos']

In [50]:
df2[stable]

,AUD_low,CAD_low,CHF_low,EUR_low,GBP_low,JPY_low,NZD_low,USD_low,AUD_high,CAD_high,...,NZD_sharpe,USD_sharpe,month_sin,month_cos,day_sin,day_cos,weekday_sin,weekday_cos,hour_sin,hour_cos
0,-0.000002,-3.371854e-07,-0.000003,-0.000003,6.822225e-07,0.000004,-3.642670e-07,0.000003,-2.280674e-02,-4.984172e-03,...,NaN,NaN,5.000000e-01,0.866025,0.052336,0.998630,0.000000,1.00000,0.000000,1.000000
1,-0.000020,-5.931761e-06,-0.000035,-0.000040,7.999135e-06,0.000056,-2.548180e-06,0.000039,4.396958e-02,-1.387042e-02,...,NaN,NaN,5.000000e-01,0.866025,0.052336,0.998630,0.000000,1.00000,0.258819,0.965926
2,-0.000125,-4.420593e-05,-0.000240,-0.000254,4.268622e-05,0.000374,-3.325799e-06,0.000250,-2.954549e-02,9.292598e-02,...,NaN,NaN,5.000000e-01,0.866025,0.052336,0.998630,0.000000,1.00000,0.500000,0.866025
3,-0.000525,-1.899295e-04,-0.001070,-0.001054,1.327127e-04,0.001617,3.659920e-05,0.001052,-3.005826e-02,-4.140157e-02,...,NaN,NaN,5.000000e-01,0.866025,0.052336,0.998630,0.000000,1.00000,0.707107,0.707107
4,-0.001667,-5.213432e-04,-0.003529,-0.003239,2.411700e-04,0.005083,2.628464e-04,0.003370,1.053280e-01,-1.466905e-01,...,NaN,NaN,5.000000e-01,0.866025,0.052336,0.998630,0.000000,1.00000,0.866025,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111091,-0.102755,6.377106e-03,-0.083646,0.198149,2.071223e-02,-0.054881,-3.557322e-02,0.051618,1.387779e-16,1.170938e-17,...,-5.114900e+06,9.791170e+06,-2.449294e-16,1.000000,0.017452,0.999848,0.781831,0.62349,-0.965926,0.258819
111092,-0.102755,6.377106e-03,-0.083646,0.198149,2.071223e-02,-0.054881,-3.557322e-02,0.051618,1.318390e-16,1.257675e-17,...,-5.114900e+06,9.791170e+06,-2.449294e-16,1.000000,0.017452,0.999848,0.781831,0.62349,-0.866025,0.500000
111093,-0.102755,6.377106e-03,-0.083646,0.198149,2.071223e-02,-0.054881,-3.557322e-02,0.051618,1.110223e-16,9.974660e-18,...,-5.114900e+06,9.791170e+06,-2.449294e-16,1.000000,0.017452,0.999848,0.781831,0.62349,-0.707107,0.707107
111094,-0.102755,6.377106e-03,-0.083646,0.198149,2.071223e-02,-0.054881,-3.557322e-02,0.051618,1.040834e-16,2.168404e-18,...,-5.114900e+06,9.791170e+06,-2.449294e-16,1.000000,0.017452,0.999848,0.781831,0.62349,-0.500000,0.866025


In [109]:
currencies = ['AUD','CAD','EUR','CHF','JPY','GBP','USD','NZD']


training_data = None
training_predictions = None

encoder = OneHotEncoder(sparse = False)

for ccy in currencies:

    encoder = OneHotEncoder(sparse = False)
    print(df.filter(regex=ccy).applymap(lambda x : condition(x , 0.05)).shape , flush = True)
    temp_y = np.array([encoder.fit_transform(df.filter(regex=ccy).applymap(lambda x : condition(x , 0.05)))])
    temp = np.array([pd.concat([df.filter(regex=ccy) , df2[stable]] , axis = 1).to_numpy()])

    if training_data is None:
        training_data = temp
    else:
        #print(training_data.shape)
        training_data = np.concatenate([ training_data , temp ], axis = 0)

    if training_predictions is None:
        training_predictions = temp_y
    else:
        training_predictions = np.concatenate([ training_predictions , temp_y] , axis = 0)

training_data.shape , training_predictions.shape
    

(111096, 7)
(111096, 7)
(111096, 7)
(111096, 7)
(111096, 7)
(111096, 7)
(111096, 7)
(111096, 7)


((8, 111096, 55), (8, 111096, 21))

In [69]:
df

,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
0,-0.056357,0.044868,-0.236804,-0.027523,-0.191681,0.090711,-0.187507,-0.277763,-0.046156,-0.119630,...,-0.147436,0.041166,-0.099046,0.009273,0.090472,-0.190166,-0.139295,0.145630,0.228210,-0.048759
1,0.057418,0.116563,-0.162348,-0.068844,-0.031983,0.047432,-0.211379,-0.267380,-0.023087,0.033756,...,-0.180774,-0.117932,-0.052168,0.159828,0.212410,-0.054403,0.069672,0.087273,0.144556,-0.126891
2,-0.206473,0.062703,-0.262812,-0.018366,-0.243434,0.294601,-0.058796,-0.346427,0.000000,-0.215034,...,-0.255167,-0.003746,-0.240322,-0.190002,0.082882,-0.245215,-0.223152,0.033227,0.323370,-0.029303
3,-0.373553,-0.049272,0.125235,-0.404915,-0.733783,0.309505,0.504518,0.201297,0.063483,-0.320355,...,0.300982,-0.251275,-0.561166,0.052171,0.367330,0.544108,-0.307650,0.348187,0.669742,0.846226
4,-0.036356,-0.236578,-0.062598,0.101392,0.012918,-0.200263,-0.070237,0.133973,0.184438,0.191093,...,0.040733,0.270002,0.152371,-0.147307,-0.350272,-0.203694,-0.098087,-0.024836,-0.256038,-0.111437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111091,-0.111104,-0.019367,-0.046219,-0.060675,-0.152940,0.088497,0.060510,-0.024679,0.303152,0.193995,...,0.078071,0.051580,-0.035225,-0.039843,0.053424,0.017658,-0.083248,0.045766,0.133986,0.100968
111092,-0.111104,-0.019367,-0.046219,-0.060675,-0.152940,0.088497,0.060510,-0.024679,0.303152,0.193995,...,0.078071,0.051580,-0.035225,-0.039843,0.053424,0.017658,-0.083248,0.045766,0.133986,0.100968
111093,-0.111104,-0.019367,-0.046219,-0.060675,-0.152940,0.088497,0.060510,-0.024679,0.303152,0.193995,...,0.078071,0.051580,-0.035225,-0.039843,0.053424,0.017658,-0.083248,0.045766,0.133986,0.100968
111094,-0.111104,-0.019367,-0.046219,-0.060675,-0.152940,0.088497,0.060510,-0.024679,0.303152,0.193995,...,0.078071,0.051580,-0.035225,-0.039843,0.053424,0.017658,-0.083248,0.045766,0.133986,0.100968


In [110]:
training_data.shape , training_predictions.shape

((8, 111096, 55), (8, 111096, 21))

In [39]:
y_

(100096, 28)

In [121]:
def initialize_model(X , y):
    model = Sequential()
    
    model.add(LSTM(50, activation='relu' , input_shape = (X.shape[0], 1)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(50 , activation = 'relu'))
    model.add(Dense(y.shape[-1], activation='sigmoid',))


    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

X_all_expanded = np.expand_dims(X , -1)
model = initialize_model(training_data , training_predictions)

model.summary()

ValueError: Unknown activation function: sigmoud. Please ensure this object is passed to the `custom_objects` argument. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [93]:
model.fit(training_data , training_predictions , epochs = 39)

Epoch 1/100


ValueError: in user code:

    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/input_spec.py", line 264, in assert_input_compatibility
        ra

In [205]:
# LSTM with Dropout for sequence classification in the IMDB dataset
import tensorflow as tf
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Embedding
from keras.preprocessing import sequence

# fix random seed for reproducibility
tf.random.set_seed(7)

# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# train the model
model.fit(X_train, y_train, epochs=3, batch_size=64)

# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("\nAccuracy: %.2f%%" % (scores[1]*100))

17464789/17464789 [==============================] - 4s 0us/step
Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 32)           160000    
                                                                 
 dropout (Dropout)           (None, 500, 32)           0         
                                                                 
 lstm_36 (LSTM)              (None, 100)               53200     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_33 (Dense)            (None, 1)                 101       
                                                                 
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
________________________________________________